In [28]:
%run save_embeds_2.py

Loading embeddings file from ../data/embeddings/crawl-300d-2M-subword.vec
2000000
2000000
Saving binary file to ../data/embeddings/crawl-300d-2M-subword.vec.wv
Saving vocabulary file to ../data/embeddings/crawl-300d-2M-subword.vec.vocab


## Preprocessing

### Load embeddings

In [1]:
import codecs
import numpy as np
from numpy import linalg as LA

def load_embeddings_from_np(filename):
    print('loading ...')
    with codecs.open(filename + '.vocab', 'r', 'utf-8') as f_embed:
        vocab = [line.strip() for line in f_embed]
        
    w2i = {w: i for i, w in enumerate(vocab)}
    wv = np.load(filename + '.wv.npy')

    return vocab, wv, w2i


def normalize(wv):
    
    # normalize vectors
    norms = np.apply_along_axis(LA.norm, 1, wv)
    wv = wv / norms[:, np.newaxis]
    return wv


def load_and_normalize(space, filename, vocab, wv, w2i):
    vocab_muse, wv_muse, w2i_muse = load_embeddings_from_np(filename)
    wv_muse = normalize(wv_muse)
    vocab[space] = vocab_muse 
    wv[space] = wv_muse
    w2i[space] = w2i_muse
    print('done')
    

def load_wo_normalize(space, filename, vocab, wv, w2i):
    vocab_muse, wv_muse, w2i_muse = load_embeddings_from_np(filename)
    vocab[space] = vocab_muse 
    wv[space] = wv_muse
    w2i[space] = w2i_muse
    print('done')
    

In [2]:
vocab = {}
wv = {}
w2i = {}

load_and_normalize('glove_1', '../data/embeddings/glove.twitter.27B.25d.txt', vocab, wv, w2i)
load_and_normalize('glove_2', '../data/embeddings/glove.twitter.27B.50d.txt', vocab, wv, w2i)
load_and_normalize('glove_3', '../data/embeddings/glove.twitter.27B.100d.txt', vocab, wv, w2i)
load_and_normalize('glove_4', '../data/embeddings/glove.twitter.27B.200d.txt', vocab, wv, w2i)
load_and_normalize('glove_5', '../data/embeddings/glove.6B.50d.txt', vocab, wv, w2i)
load_and_normalize('glove_6', '../data/embeddings/glove.6B.100d.txt', vocab, wv, w2i)
load_and_normalize('glove_7', '../data/embeddings/glove.6B.200d.txt', vocab, wv, w2i)
load_and_normalize('glove_8', '../data/embeddings/glove.6B.300d.txt', vocab, wv, w2i)
load_and_normalize('glove_9', '../data/embeddings/glove.42B.300d.txt', vocab, wv, w2i)
load_and_normalize('glove_10', '../data/embeddings/glove.840B.300d.txt', vocab, wv, w2i)
load_and_normalize('fasttext_1', '../data/embeddings/wiki-news-300d-1M.vec', vocab, wv, w2i)
load_and_normalize('fasttext_2', '../data/embeddings/wiki-news-300d-1M-subword.vec', vocab, wv, w2i)
load_and_normalize('fasttext_3', '../data/embeddings/crawl-300d-2M.vec', vocab, wv, w2i)
load_and_normalize('fasttext_4', '../data/embeddings/crawl-300d-2M-subword.vec', vocab, wv, w2i)
load_and_normalize('w2v', '../data/embeddings/orig_w2v', vocab, wv, w2i)

loading ...
done
loading ...
done
loading ...
done
loading ...
done
loading ...
done
loading ...
done
loading ...
done
loading ...
done
loading ...
done
loading ...
done
loading ...
done
loading ...
done
loading ...
done
loading ...
done
loading ...
done


In [9]:
from tqdm import tqdm

def topK(w, space, k=10):
    
    # extract the word vector for word w
    idx = w2i[space][w]
    vec = wv[space][idx, :]
    
    # compute similarity of w with all words in the vocabulary
    sim = wv[space].dot(vec)
    # sort similarities by descending order
    sort_sim = (sim.argsort())[::-1]

    # choose topK
    best = sort_sim[:(k+1)]

    return [vocab[space][i] for i in best if i!=idx]


def similarity(w1, w2, space):
    
    i1 = w2i[space][w1]
    i2 = w2i[space][w2]
    vec1 = wv[space][i1, :]
    vec2 = wv[space][i2, :]

    return np.inner(vec1,vec2)



### Restrict vocabulary

In [16]:
import string 


def has_punct(w):
    
    if any([c in string.punctuation for c in w]):
        return True
    return False

def has_digit(w):
    
    if any([c in '0123456789' for c in w]):
        return True
    return False

def limit_vocab(space, exclude = None, vec_len):
    vocab_limited = []
    for w in tqdm(vocab[space][:50000]): 
        if w.lower() != w:
            continue
        if len(w) >= 20:
            continue
        if has_digit(w):
            continue
        if '_' in w:
            p = [has_punct(subw) for subw in w.split('_')]
            if not any(p):
                vocab_limited.append(w)
            continue
        if has_punct(w):
            continue
        vocab_limited.append(w)
    
    if exclude:
        vocab_limited = list(set(vocab_limited) - set(exclude))
    
    print("size of vocabulary:", len(vocab_limited))
    
    wv_limited = np.zeros((len(vocab_limited), vec_len))
    for i,w in enumerate(vocab_limited):
        wv_limited[i,:] = wv[space][w2i[space][w],:]
    
    w2i_limited = {w: i for i, w in enumerate(vocab_limited)}
    
    return vocab_limited, wv_limited, w2i_limited



SyntaxError: non-default argument follows default argument (<ipython-input-16-29c426f98d21>, line 16)

In [17]:
import string 


def has_punct(w):
    
    if any([c in string.punctuation for c in w]):
        return True
    return False

def has_digit(w):
    
    if any([c in '0123456789' for c in w]):
        return True
    return False

def limit_vocab(vec_len, space, exclude = None):
    vocab_limited = []
    for w in tqdm(vocab[space][:50000]): 
        if w.lower() != w:
            continue
        if len(w) >= 20:
            continue
        if has_digit(w):
            continue
        if '_' in w:
            p = [has_punct(subw) for subw in w.split('_')]
            if not any(p):
                vocab_limited.append(w)
            continue
        if has_punct(w):
            continue
        vocab_limited.append(w)
    
    if exclude:
        vocab_limited = list(set(vocab_limited) - set(exclude))
    
    print("size of vocabulary:", len(vocab_limited))
    
    wv_limited = np.zeros((len(vocab_limited), vec_len))
    for i,w in enumerate(vocab_limited):
        wv_limited[i,:] = wv[space][w2i[space][w],:]
    
    w2i_limited = {w: i for i, w in enumerate(vocab_limited)}
    
    return vocab_limited, wv_limited, w2i_limited




In [22]:
# create the reduced vocabularies and embeddings before and after, without gendered specific words

import json
with codecs.open('../data/lists/gender_specific_full.json') as f:
    gender_specific = json.load(f)
with codecs.open('../data/lists/definitional_pairs.json') as f:
    definitional_pairs = json.load(f)
with codecs.open('../data/lists/equalize_pairs.json') as f:
    equalize_pairs = json.load(f)

exclude_words = []
for pair in definitional_pairs + equalize_pairs:
    exclude_words.append(pair[0])
    exclude_words.append(pair[1])

exclude_words = list(set(exclude_words).union(set(gender_specific)))

# create spaces of limited vocabulary
vocab['limit_glove_1'], wv['limit_glove_1'], w2i['limit_glove_1'] = limit_vocab(25, 'glove_1', exclude = exclude_words)
vocab['limit_glove_2'], wv['limit_glove_2'], w2i['limit_glove_2'] = limit_vocab(50, 'glove_2', exclude = exclude_words)
vocab['limit_glove_3'], wv['limit_glove_3'], w2i['limit_glove_3'] = limit_vocab(100, 'glove_3', exclude = exclude_words)
vocab['limit_glove_4'], wv['limit_glove_4'], w2i['limit_glove_4'] = limit_vocab(200, 'glove_4', exclude = exclude_words)
vocab['limit_glove_5'], wv['limit_glove_5'], w2i['limit_glove_5'] = limit_vocab(50, 'glove_5', exclude = exclude_words)
vocab['limit_glove_6'], wv['limit_glove_6'], w2i['limit_glove_6'] = limit_vocab(100, 'glove_6', exclude = exclude_words)
vocab['limit_glove_7'], wv['limit_glove_7'], w2i['limit_glove_7'] = limit_vocab(200, 'glove_7', exclude = exclude_words)
vocab['limit_glove_8'], wv['limit_glove_8'], w2i['limit_glove_8'] = limit_vocab(300, 'glove_8', exclude = exclude_words)
vocab['limit_glove_9'], wv['limit_glove_9'], w2i['limit_glove_9'] = limit_vocab(300, 'glove_9', exclude = exclude_words)
vocab['limit_glove_10'], wv['limit_glove_10'], w2i['limit_glove_10'] = limit_vocab(300, 'glove_10', exclude = exclude_words)

vocab['limit_fasttext_1'], wv['limit_fasttext_1'], w2i['limit_fasttext_1'] = limit_vocab(300, 'fasttext_1', exclude = exclude_words)
vocab['limit_fasttext_2'], wv['limit_fasttext_2'], w2i['limit_fasttext_2'] = limit_vocab(300, 'fasttext_2', exclude = exclude_words)
vocab['limit_fasttext_3'], wv['limit_fasttext_3'], w2i['limit_fasttext_3'] = limit_vocab(300, 'fasttext_3', exclude = exclude_words)
vocab['limit_fasttext_4'], wv['limit_fasttext_4'], w2i['limit_fasttext_4'] = limit_vocab(300, 'fasttext_4', exclude = exclude_words)

vocab['limit_w2v'], wv['limit_w2v'], w2i['limit_w2v'] = limit_vocab(300, 'w2v', exclude = exclude_words)


100%|██████████| 50000/50000 [00:00<00:00, 455818.38it/s]


size of vocabulary: 49288
size of vocabulary: 49288


100%|██████████| 50000/50000 [00:00<00:00, 446536.03it/s]

size of vocabulary: 49288



  0%|          | 0/50000 [00:00<?, ?it/s]

...

size of vocabulary: 23097
size of vocabulary: 26800


100%|██████████| 50000/50000 [00:00<00:00, 622183.98it/s]


size of vocabulary: 26800
size of vocabulary: 27174


100%|██████████| 50000/50000 [00:00<00:00, 600268.48it/s]


size of vocabulary: 27231
size of vocabulary: 26189


### Compute bias-by-projection

In [23]:
# create a dictionary of the bias, before and after

def compute_bias_by_projection(space_to_tag, full_space):
    males = wv[space_to_tag].dot(wv[full_space][w2i[full_space]['he'],:])
    females = wv[space_to_tag].dot(wv[full_space][w2i[full_space]['she'],:])
    d = {}
    for w,m,f in zip(vocab[space_to_tag], males, females):
        d[w] = m-f
    return d

# compute bias-by-projection before and after debiasing
gender_bias_glove_1 = compute_bias_by_projection('limit_glove_1', 'glove_1')
gender_bias_glove_2 = compute_bias_by_projection('limit_glove_2', 'glove_2')
gender_bias_glove_3 = compute_bias_by_projection('limit_glove_3', 'glove_3')
gender_bias_glove_4 = compute_bias_by_projection('limit_glove_4', 'glove_4')
gender_bias_glove_5 = compute_bias_by_projection('limit_glove_5', 'glove_5')
gender_bias_glove_6 = compute_bias_by_projection('limit_glove_6', 'glove_6')
gender_bias_glove_7 = compute_bias_by_projection('limit_glove_7', 'glove_7')
gender_bias_glove_8 = compute_bias_by_projection('limit_glove_8', 'glove_8')
gender_bias_glove_9 = compute_bias_by_projection('limit_glove_9', 'glove_9')
gender_bias_glove_10 = compute_bias_by_projection('limit_glove_10', 'glove_10')

gender_bias_fasttext_1 = compute_bias_by_projection('limit_fasttext_1', 'fasttext_1')
gender_bias_fasttext_2 = compute_bias_by_projection('limit_fasttext_2', 'fasttext_2')
gender_bias_fasttext_3 = compute_bias_by_projection('limit_fasttext_3', 'fasttext_3')
gender_bias_fasttext_4 = compute_bias_by_projection('limit_fasttext_4', 'fasttext_4')

gender_bias_w2v = compute_bias_by_projection('limit_w2v', 'w2v')


In [37]:
gender_bias_glove_1

{'intelligent': -0.04272356091898766,
 'hitting': -0.0957484362985912,
 'др': 0.046130826205270925,
 'دائماً': 0.0315981558892493,
 'sciences': 0.007764622479345326,
 'fast': -0.026675347271595817,
 'awsome': -0.16704667146859226,
 'blijft': 0.11334647729971739,
 'حسنات': -0.06571975978277511,
 'qaqa': -0.004883174395591064,
 'sleeps': -0.18127195148185182,
 'boring': -0.0829141994615572,
 'طلبت': 0.01909266587099201,
 'krmh': -0.06129025917847195,
 'jurado': 0.1953882791442411,
 'rants': -0.24785945891633793,
 'sengaja': 0.012843533467377821,
 'capire': 0.0391002215248404,
 'uw': 0.03154366637031253,
 'compton': -0.16089035386948575,
 'hazard': 0.14765659627478517,
 'tsc': -0.04244466034627097,
 'وأبـعـد': -0.11401281665115967,
 'chuck': -0.004147669434868773,
 'hardcore': -0.11892286463949131,
 'teknik': 0.03283581981135647,
 'صباح_الخير': -0.2037765955260586,
 'daq': 0.01565622792586536,
 'وضيف': -0.11253097562307095,
 '✔': -0.11144216281242203,
 'ultima': 0.18766829647300476,
 'дек

In [24]:
# calculate the avg bias of the vocabulary (abs) before and after debiasing

def report_bias(gender_bias):
    bias = 0.0
    for k in gender_bias:
        bias += np.abs(gender_bias[k])
    print(bias/len(gender_bias))

#### GloVe Twitter

In [25]:
report_bias(gender_bias_glove_1)
report_bias(gender_bias_glove_2)
report_bias(gender_bias_glove_3)
report_bias(gender_bias_glove_4)

0.09540471479449833
0.08798050874134224
0.07545063847129793
0.062368382907960776


#### GloVe Wiki Gigaword

In [26]:
report_bias(gender_bias_glove_5)
report_bias(gender_bias_glove_6)
report_bias(gender_bias_glove_7)
report_bias(gender_bias_glove_8)

0.06009518969994554
0.047896221138184286
0.045611368866501315
0.04346563770051536


#### GloVe Common Crawl

In [27]:
report_bias(gender_bias_glove_9)
report_bias(gender_bias_glove_10)

0.030484643981032414
0.0390976826352134


#### word2vec

In [28]:
report_bias(gender_bias_w2v)

0.04607731006352736


#### fastText Wiki News

In [29]:
report_bias(gender_bias_fasttext_1)
report_bias(gender_bias_fasttext_2)

0.026961478262307156
0.02273574827560833


#### fastText Common Crawl

In [30]:
report_bias(gender_bias_fasttext_3)
report_bias(gender_bias_fasttext_4)

0.03212379572532563
0.022733283621889155


## Experiments
### Bias-by-neighbours

In [31]:
# get tuples of biases and counts of masculine/feminine NN for each word (for bias-by-neighbors)

def bias_by_neighbors(gender_bias, space, neighbours_num = 100):
    
    tuples = []
    for w in tqdm(vocab[space]):
        
        top = topK(w, space, k=neighbours_num+5)[:neighbours_num]

        m = 0
        f = 0    
        for t in top:
            if gender_bias[t] > 0:
                m+=1
            else:
                f+=1

        tuples.append((w, gender_bias[w], m, f))

    return tuples

In [32]:
tuples_glove_1 = bias_by_neighbors(gender_bias_glove_1, 'limit_glove_1')   
tuples_glove_2 = bias_by_neighbors(gender_bias_glove_2, 'limit_glove_2')  
tuples_glove_3 = bias_by_neighbors(gender_bias_glove_3, 'limit_glove_3')  
tuples_glove_4 = bias_by_neighbors(gender_bias_glove_4, 'limit_glove_4')  
tuples_glove_5 = bias_by_neighbors(gender_bias_glove_5, 'limit_glove_5')  
tuples_glove_6 = bias_by_neighbors(gender_bias_glove_6, 'limit_glove_6')  
tuples_glove_7 = bias_by_neighbors(gender_bias_glove_7, 'limit_glove_7')
tuples_glove_8 = bias_by_neighbors(gender_bias_glove_8, 'limit_glove_8')  
tuples_glove_9 = bias_by_neighbors(gender_bias_glove_9, 'limit_glove_9')  
tuples_glove_10 = bias_by_neighbors(gender_bias_glove_10, 'limit_glove_10')  

tuples_w2v = bias_by_neighbors(gender_bias_w2v, 'limit_w2v')  

tuples_fasttext_1 = bias_by_neighbors(gender_bias_fasttext_1, 'limit_fasttext_1')  
tuples_fasttext_2 = bias_by_neighbors(gender_bias_fasttext_2, 'limit_fasttext_2')  
tuples_fasttext_3 = bias_by_neighbors(gender_bias_fasttext_3, 'limit_fasttext_3')  
tuples_fasttext_4 = bias_by_neighbors(gender_bias_fasttext_4, 'limit_fasttext_4')  


100%|██████████| 27231/27231 [01:38<00:00, 275.95it/s]


### Correlation between bias-by-projection and bias-by-neighbors

In [ ]:
#compute correlation between bias-by-projection and bias-by-neighbors

import scipy.stats

def pearson(a,b):
   
    return scipy.stats.pearsonr(a,b)

def compute_corr(tuples, i1, i2):
    
    a = []
    b = []
    for t in tuples:
        a.append(t[i1])
        b.append(t[i2])
    assert(len(a)==len(b))    
    print(pearson(a,b))


#### GloVe Twitter

In [62]:
compute_corr(tuples_glove_1, 1, 2)
compute_corr(tuples_glove_2, 1, 2)
compute_corr(tuples_glove_3, 1, 2)
compute_corr(tuples_glove_4, 1, 2)

(0.8724100178362321, 0.0)
(0.8586076557038413, 0.0)
(0.8497446347007468, 0.0)
(0.8347333503930401, 0.0)


#### GloVe Wiki Gigaword

In [63]:
compute_corr(tuples_glove_5, 1, 2)
compute_corr(tuples_glove_6, 1, 2)
compute_corr(tuples_glove_7, 1, 2)
compute_corr(tuples_glove_8, 1, 2)

(0.8442397459468596, 0.0)
(0.8119382402476751, 0.0)
(0.7730547290513069, 0.0)
(0.7444402487664554, 0.0)


#### GloVe Common Crawl

In [64]:
compute_corr(tuples_glove_9, 1, 2)
compute_corr(tuples_glove_10, 1, 2)

(0.7697368579080862, 0.0)
(0.7702136554401371, 0.0)


#### word2vec

In [58]:
compute_corr(tuples_w2v, 1, 2)

(0.7406908013529686, 0.0)


#### fastText Wiki News

In [59]:
compute_corr(tuples_fasttext_1, 1, 2)
compute_corr(tuples_fasttext_2, 1, 2)

(0.6290830393062714, 0.0)
(0.6249758074699922, 0.0)


#### fastText Common Crawl

In [60]:
compute_corr(tuples_fasttext_3, 1, 2)
compute_corr(tuples_fasttext_4, 1, 2)

(0.6336795662445177, 0.0)
(0.6200156403337626, 0.0)
